In [1]:
%%prun
import glob
import json
from pathlib import Path
from CustomDatasetPytorch import CustomAllLoadDataset
import torch
import time
 # Get the path to the config file
experiments_folder = "C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression"
task_folder = Path(experiments_folder)
config_path = task_folder/ "util/config.json"
    
with open(config_path) as fp:
    config = json.load(fp)

data_folder = Path(config["dataset_folder"])/ config["split_folder"]
stimulus_features = ["envelope"]
features = ["eeg"] + stimulus_features

train_files = [path for path in Path(data_folder).resolve().glob("train_-_*") if path.stem.split("_-_")[-1].split(".")[0] in features]

train_dataset = CustomAllLoadDataset(train_files,64*3,int(64*1.5))
train_dataset.convertToTensorType()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_dataset.send_to_device(device,0.7)

dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)

start_time = time.time()
for ite in range(1):
    
    count = 0
    for batch in dataloader:
        count+=1
print("total: ", (count-1)*64,batch[0].shape,batch[1].shape)
print("--- %s seconds ---" % (time.time() - start_time))

508  files loaded.  229547  training examples loaded
total:  229504 torch.Size([43, 64, 192]) torch.Size([43, 1, 192])
--- 479.3682773113251 seconds ---
 

In [3]:
import torch
from pathlib import Path
import itertools
import numpy as np
import time
import random
from random import randrange
import torchaudio.transforms as T
import torch

class CustomAllLoadRawWaveDataset(torch.utils.data.Dataset):
    def __init__(self, filePaths, frameLength, hopSize, resample_rate):
        self.filePaths = filePaths
        self.frameLength = frameLength
        self.hopSize = hopSize
        self.resample_rate = resample_rate
        self.filePage = len(self.filePaths)
        # load all data to memory as continuous data
        self.AudioData = []
        self.loadDataToBuffer()

        self.sampleIndxMap = []
        self.noData = self.generateSamplePostion()
        print("data load finished: ", self.filePage, " files in total ", self.noData, " samples in total")

    def loadDataToBuffer(self):
        for i in range(self.filePage):
            b = np.load(self.filePaths[i])
            data = torch.from_numpy(b['audio'])
            sample_rate = b['fs']
            resampler = T.Resample(sample_rate, self.resample_rate, dtype=data.dtype)
            resampled_waveform = resampler(data)
            print("file ", i, "from: ", data.shape[0], "to ", resampled_waveform.shape[0])
            self.AudioData.append(resampled_waveform)

    def generateSamplePostion(self):
        count = 0
        for i in range(self.filePage):
            totalLength = self.AudioData[i].shape[0]
            startPos = [*range(self.frameLength, totalLength + 1, self.hopSize)]
            for pos in startPos:
                self.sampleIndxMap.append((i, pos))
            noData = (totalLength - self.frameLength) // self.hopSize + 1
            assert len(startPos) == noData
            print("file ", i, "load: ", noData)
            count += noData
        return count

    def __len__(self):
        return self.noData

    def __getitem__(self, idx):
        return np.array([[[0], [1], [2]]]), torch.unsqueeze(self.AudioData[self.sampleIndxMap[idx][0]][self.sampleIndxMap[idx][1] - self.frameLength:self.sampleIndxMap[idx][1]],0)

In [ ]:
import json

# Load the config
with open("C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression/util/config.json") as fp:
    config = json.load(fp)

# Provide the path of the dataset
# which is split already to train, val, test

data_folder = Path(config["dataset_folder"])/config["raw_stimuli_folder"]
train_files = [path for path in Path(data_folder).resolve().glob("*.npz") if path.stem.split("_")[1] == "1"]
print(len(train_files))
b = np.load(train_files[3])
data = torch.from_numpy(b['audio'])
sample_rate = b['fs']
resampler = T.Resample(sample_rate, 11600, dtype=data.dtype)
resampled_waveform = resampler(data)

In [ ]:
from IPython.display import Audio

Audio(resampled_waveform, rate=11600)

In [ ]:
data = train_dataset.__getitem__(2300)[1]

In [ ]:
from IPython.display import Audio

Audio(data, rate=11600)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)

plot_a = plt.subplot(211)
plot_a.plot(data.squeeze())
plot_a.set_xlabel('sample rate * time')
plot_a.set_ylabel('energy')

plot_b = plt.subplot(212)
plot_b.specgram(data.squeeze(), NFFT=1024, Fs=11600, noverlap=512)
plot_b.set_xlabel('Time')
plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
import json

# Load the config
with open("C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression/util/config.json") as fp:
    config = json.load(fp)

# Provide the path of the dataset
# which is split already to train, val, test

data_folder = Path(config["dataset_folder"])/config["raw_stimuli_folder"]
train_files = [path for path in Path(data_folder).resolve().glob("*.npz") if path.stem.split("_")[1] == "1"]
print(train_files[0])
b = np.load(train_files[0])
data = torch.from_numpy(b['audio'])
print(data.shape)
print(b['fs'])
sample_rate = b['fs']
resampler = T.Resample(sample_rate, 11600, dtype=data.dtype)
resampled_waveform = resampler(data)

In [ ]:
import json

# Load the config
with open("C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression/util/config.json") as fp:
    config = json.load(fp)

# Provide the path of the dataset
# which is split already to train, val, test

data_folder = Path(config["dataset_folder"])/config["raw_eeg_folder"]
train_files = [path for path in Path(data_folder).resolve().glob("*.pkl")]
print(len(train_files))
import pickle


with open(train_files[2], 'rb') as f:
    data = pickle.load(f)
    print(data['eeg'].shape)
    print(data['fs'])
    print(data['subject'])
    print(data['stimulus'])
    print(len(data))

In [ ]:
print(data['eeg'].shape)
print(data['fs'])
print(data['subject'])
print(data['stimulus'])
print(len(data))